In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import os

In [2]:
# get appdata path to this path on any commputer: "C:/Users/Domodekavkaz/AppData/Roaming/Holovibes/12.10.0/benchmark/
appdata = os.getenv('APPDATA')
path = appdata + "/Holovibes/12.10.0/benchmark/"
print(path)

C:\Users\Voronezh\AppData\Roaming/Holovibes/12.10.0/benchmark/


In [3]:
path += "benchmark_2024-04-25_10h52-36.csv"

In [4]:
df = pd.read_csv(path, header=1)

In [5]:
# print the list of columns
print(df.columns)

Index(['Input Queue', 'Output Queue', 'Record Queue', 'Input FPS',
       'Output FPS', 'Input Throughput', 'Output Throughput',
       'GPU memory free', 'GPU memory total', 'GPU load', 'GPU memory load',
       ' z_boundary'],
      dtype='object')


In [6]:
# Afficher les premières lignes du DataFrame
print(df.head())

# Extraire les métadonnées du fichier CSV
with open(path, 'r') as file:
    first_line = file.readline()
    metadata = first_line.split(',')

print("Métadonnées :")
for meta in metadata:
    print(meta)


   Input Queue  Output Queue  Record Queue  Input FPS  Output FPS  \
0            5             0             0          0           0   
1            9             0             0          0           0   
2           13             0             0          0           0   
3           17             0             0         31           0   
4           21             0             0         31           0   

   Input Throughput  Output Throughput  GPU memory free  GPU memory total  \
0                 0                  0      23626764288       25756696576   
1                 0                  0      23607889920       25756696576   
2                 0                  0      23584821248       25756696576   
3           9523200                  0      23557558272       25756696576   
4           9523200                  0      23530295296       25756696576   

   GPU load  GPU memory load   z_boundary  
0        17                1     0.090226  
1         6                0     0

In [7]:
# make a dictionary of the metadata, for each element of metadata separated into two part from ': ', the key is the first part, and the value is the second part

metadata_dict = {}
for meta in metadata:
    key, value = meta.split(': ')
    metadata_dict[key] = value

In [8]:
# print the list of columns
print(df.index)

RangeIndex(start=0, stop=288, step=1)


In [9]:
# use plotly to create a line chart that has each "Queue" column as a line, and the x-axis is the length of the index
fig = px.line(df, x=range(len(df)), y=['Input Queue', 'Output Queue', 'Record Queue'])
# add à horizontal blue line at the level of the "GPU input queue size" metadata
fig.add_hline(y=int(metadata_dict['GPU Input Queue size']), line_dash="dot", line_color="blue", annotation_text="GPU input queue size", annotation_position="bottom right")
# add à horizontal red line at the level of the "GPU output queue size" metadata
fig.add_hline(y=int(metadata_dict['GPU Output Queue size']), line_dash="dot", line_color="red", annotation_text="GPU output queue size", annotation_position="bottom right")
# add à horizontal green line at the level of the "GPU record queue size" metadata
fig.add_hline(y=int(metadata_dict['CPU Record Queue size']), line_dash="dot", line_color="green", annotation_text="CPU record queue size", annotation_position="bottom right")

fig.show()

In [10]:
# use plotly to create a line chart that has each "FPS" column as a line, and the x-axis is the length of the index
fig = px.line(df, x=range(len(df)), y=['Output FPS'])
# add à horizontal blue line at the level of the "Input FPS" first value
fig.add_hline(y=df['Input FPS'][20], line_dash="dot", line_color="blue", annotation_text="Input FPS", annotation_position="bottom right")

fig.show()


In [11]:
# use plotly to create a line chart that has each "Throughput" column as a line, and the x-axis is the length of the index
fig = px.line(df, x=range(len(df)), y=['Input Throughput', 'Output Throughput'])

fig.show()


In [12]:
# Create a line chart with each "GPU" column as a line
fig = px.line(df, x=range(len(df)), y=['GPU load', 'GPU memory load'])

# Add a line with the division of the GPU Load by the GPU Memory Load
fig.add_trace(px.line(df, x=range(len(df)), y=((df['GPU memory total'] - df['GPU memory free'])/df['GPU memory total']) * 100).data[0])

# Update the name of the added trace
fig.data[-1].name = 'GPU Memory division'

# Change the color of the added line to purple
fig.update_traces(line_color='purple', selector=dict(name='GPU Memory division'))

# Show the plot
fig.show()